# Representación de palabras mediante espacios vectoriales

## Créditos

Estas clases y material están fuertemente inspirado en los cursos de Stanford [CS224n: Natural Language Processing with Deep Learning](http://web.stanford.edu/class/cs224n/) y [CS224u: Natural Language Understanding](http://web.stanford.edu/class/cs224u/).

In [1]:
__author__ = "Cristian Cardellino"

## Contenidos

1. [Representación de palabras](#Representación-de-palabras)
1. [Representación distribuída de palabras](#Representación-distribuída-de-palabras)
1. Diseño de matrices
    1. Matriz de co-ocurrencia de palabras
    1. Matriz de palabra-vector
    1. Otros tipos de matrices
    1. Ventana y ajuste
1. Comparación de vectores
    1. Distancia euclídea
    1. Normalización de longitud
    1. Distancia (y similitud) coseno
    1. Otras medidas
    1. Propiedades de una distancia
    1. Generalidades
1. Métodos de reponderación
    1. Objetivos de reponderar
    1. Normalización
    1. Observado/Esperado
    1. Información mutua
    1. TF-IDF
    1. Generalidades
1. Visualización
    1. PCA
    1. t-SNE
1. Reducción de dimensionalidad
    1. LSA
    1. Autoencoders
    1. word2vec
    1. GloVe

## Representación de palabras

1. [Características de las palabras](#Características-de-las-palabras)
1. [Representación mediante taxonomías](#Representación-mediante-taxonomías)
1. [Representación discreta](#Representación-discreta)
1. [Ejemplo de representación discreta](#Ejemplo-de-representación-discreta)

### Características de las palabras

- Son la **unidad básica** en cualquier tarea de procesamiento de lenguaje natural (PLN).
- Cualquier modelo de PLN requiere como entrada la **representación de una palabra**.
- Gran parte del trabajo de PLN solía representar las palabras como **símbolos atómicos**.
- Las nociones de **similitu y distancia** entre palabras son cruciales para tareas de PLN.

### Representación mediante taxonomías

- El **sentido** de una palabras es la idea que una palabra (o frase) representa.
- Se puede representar mediante recursos del estilo de las **taxonomías**.
- Ofrecen **información rica y estructurada**.
    - WordNet asigna relaciones de **hiperonimia/hiponimia** (relación *es-un*), y **sinonimia**.
    - FrameNet establece la **estructura** mediante un marco sintáctico y semántico de las palabras.
- Son **difíciles y caros de obtener y actualizar** (requiren anotación humana).

In [2]:
from nltk.corpus import wordnet as wn  # Run nltk.download("wordnet") first

dog = wn.synset("dog.n.01")

print("Dog's hypernyms:")
for hypernym in dog.hypernyms():
    print(hypernym)

Dog's hypernyms:
Synset('canine.n.02')
Synset('domestic_animal.n.01')


### Representación discreta

- Para aplicar medidas de similitud y distancia (e.g. euclídea, coseno), utilizamos **vectores que representen palabras**.
- Primera aproximación: **vectores *one-hot***.
- Para cada palabra del vocabulario, tenemos un vector.
- En términos de espacio vectorial, el vector tiene un **1** en una de sus dimensiones y __0s__ en todas las demás.
- **Dimensionalidad muy alta** (el inglés tiene un estimado de 13 millones de palabras).
- Los vectores son **ralos** (o esparsos), tienen muchos ceros.

### Ejemplo de representación discreta

Dado un corpus:

- "El fallo fue decisivo"
- "La corte rectificará el fallo"

Tenemos un vocabulario $V = \{corte, decisivo, el, fallo, fue, la, rectificará\}$.

Codificamos un número $|V| = 7$ de vectores para cada palabra $w^{(i)} \in \mathbb{R}^{|V|}$.

### Ejemplo de representación discreta

$$
    w^{corte} =
    \begin{bmatrix}
       1 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0
    \end{bmatrix} ,
    w^{decisivo} =
    \begin{bmatrix}
       0 \\ 1 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0
    \end{bmatrix} ,
    w^{el} =
    \begin{bmatrix}
       0 \\ 0 \\ 1 \\ 0 \\ 0 \\ 0 \\ 0
    \end{bmatrix} ,
    w^{fallo} =
    \begin{bmatrix}
       0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0 \\ 0
    \end{bmatrix} ,
$$

$$
    w^{fue} =
    \begin{bmatrix}
       0 \\ 0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0
    \end{bmatrix}
    w^{la} =
    \begin{bmatrix}
       0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 1 \\ 0
    \end{bmatrix} ,
    w^{rectificará} =
    \begin{bmatrix}
       0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 1
    \end{bmatrix}
$$

In [3]:
import numpy as np

corpus = [
    "El fallo fue decisivo", 
    "La corte rectificará el fallo"
]

tokenized_corpus = [
    [word.lower() for word in doc.split()] for doc in corpus
]

vocabulary = sorted(set([
    word for doc in tokenized_corpus for word in doc
]))

one_hot_vectors = np.eye(len(vocabulary), dtype=np.int)

for doc in tokenized_corpus:
    print(np.array([
        one_hot_vectors[vocabulary.index(word), :] for word in doc
    ]))

[[0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0]]
[[0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0]]


## Representación distribuída de palabras

1. Objetivos de alto nivel
1. Hipótesis sobre representación distribuída